#### [미션] 여러 page 데이터 가져오기(지식인 검색)
- 사용자로부터 검색어, 페이지 입력
- 검색어에 대한 답변 추출 => csv 형식으로 데이터 저장
- 저장내용: 검색어/등록일/답변


##### 작업순서
1. url 구성: 기본url + 검색어 + 페이지(변경)
2. 기본 URL 추출
3. 기본 URL을 이용한 답변 추출
4. DataFrame으로 변경후 검색어.csv로 저장

1. 사용자에게 검색어와 페이지 건수 입력 및 url 추출

In [76]:
import requests
import pandas as pd
import tqdm
from bs4 import BeautifulSoup as bs

In [77]:
# url 생성
sch=input('검색어 입력:')
pages=input('종료페이지 입력:')

url_list=[]

for page in tqdm.tqdm(range(1, int(pages)+1)):
    url = f'https://kin.naver.com/search/list.naver?query={sch}&page={page}'
    html = requests.get(url)
    if html.status_code != 200:continue
    soup = bs(html.text, 'html.parser') 
    lis = soup.select('ul.basic1 li')
    for li in lis:
        url_list.append(li.find('a')['href'])

print("\n",len(url_list))

검색어 입력:부산 서면
종료페이지 입력:2


100%|██████████| 2/2 [00:01<00:00,  1.83it/s]


 20


2. 추출한 URL을 이용해 답변 추출

In [78]:
answer_list=[]

for url in tqdm.tqdm(url_list):
    # 정보 요청
    html = requests.get(url)
    if html.status_code != 200: # 정보 오류 여부 확인
        continue

    # 정보 추출
    soup = bs(html.text, 'html.parser') 
    anscnt=soup.find('em', class_="_answerCount num").text
    if int(anscnt) < 1:
        continue  
    #print(anscnt)

    # 답변 텍스트 추출
    ques_date=soup.select('div.question-content span.c-userinfo__info')[0].text
    title=soup.select('div.question-content div.title')[0].text
    try:
        question=soup.select('div.question-content div.c-heading__content')[0].text
    except:
        question="내용없음"
    answers=soup.select('div.answer-content div.se-main-container')

    # 데이터 추가
    for answer in answers:
        answer_list.append({'ques_date':ques_date.replace('작성일',""),
                        'title':title.replace('\n',"").replace('\t',""),
                        'question':question.replace('\n',"").replace('\t',""),
                        'answer':answer.text.replace('\n',"").replace('\t',"")})

100%|██████████| 20/20 [00:10<00:00,  1.92it/s]


3. 데이터프레임 형식으로 변경후 csv 저장

In [79]:
df=pd.DataFrame(answer_list)
df

,ques_date,title,question,answer
0,2022.12.22,내일 부산 서면 날씨,내용없음,부산지방기상청 1시간관측 현재날씨 예보 12.22.(목) 23:0...
1,2022.12.22,내일 부산 서면 날씨,내용없음,내일 낮 최고기온이 체감온도 기준으로 2도밖에 되지 않아서 많이 춥겠습니다...
2,2023.01.02,전주에서 부산 서면까지가고싶습니다 내공80,이번에 친구들가 부산으로놀러가려고 하는데 전주에서 부산서면까지 가는 저렴한방법있을까요?,​​​사상역에서 2호선 탑승 서면역 하차 ...
3,2023.01.02,전주에서 부산 서면까지가고싶습니다 내공80,이번에 친구들가 부산으로놀러가려고 하는데 전주에서 부산서면까지 가는 저렴한방법있을까요?,​전주터미널에서 부산서부터미널까지 시외버스 이용 하시면 됩니다부산서부터미널...
4,2023.01.02,전주에서 부산 서면까지가고싶습니다 내공80,이번에 친구들가 부산으로놀러가려고 하는데 전주에서 부산서면까지 가는 저렴한방법있을까요?,전주터미널에서 부산서부터미널까지 시외버스 이용 하시면 됩니다부산서부터미널에...
5,2022.08.22,부산 서면 원룸시세,서면 원룸 보증금 500에 월세 얼마정도 할까요??,안녕하세요~^^​작은 도움이 되길 바라며 답변드립니다.​1. 서면 원룸 보...
6,2022.08.22,부산 서면 원룸시세,서면 원룸 보증금 500에 월세 얼마정도 할까요??,안녕하세요. 다방입니다.부산 서면 원룸시세에 대해 궁금하시군요. 다방이 도...
7,2022.07.26,부산 서면에서 가까운 버스 터미널,부산 서면에 놀러 갈 예정인데 다 놀고 시외버스 타려는데서면에서 제일 가까운 시외버...,▶ 서면역은 부산에서 보면 중앙에 위치하고 있습니다. - 지하철2호선과...
8,2022.07.26,부산 서면에서 가까운 버스 터미널,부산 서면에 놀러 갈 예정인데 다 놀고 시외버스 타려는데서면에서 제일 가까운 시외버...,"부산종합버스터미널, 부산서부시외버스터미널입니다(후자가 지하철 8정거장정도 ..."
9,2022.12.28,부산 서면 시세,안녕하세요 서울토박이인데용 부산 갈 일이 생겨서 원1룸 알아보는데 서울에만 살았어서...,원래 그렇습니다 :D !허위매물이 그리 많은 편은 아녜요!!​서면으로 거주...


In [82]:
df.to_csv(sch+'.csv', index=False,encoding='utf-8-sig')